# Traning RNN on Simulation

### Imports

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

D:\Programme\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



2.15.0


In [276]:
import random
import math
import pickle
import Simulation.KNNSim as sim

### Load data and helper tables

In [3]:
startIndexes = pickle.load(open('Simulation/E0F1_index_transmision.pkl', 'rb'))
usablePVindextable = pickle.load(open('Simulation/E0F1_index_I1.pkl', 'rb'))
pvNames = pickle.load(open('Simulation/E0F1Names.pkl', 'rb'))

### Define the shape of the input...

In [362]:
input_dim =  len(usablePVindextable)*2 + 1 # permissionbools for PVs (dependand on subsection),current PV values , current current value
output_dim = len(usablePVindextable)
input_dim

25

### Define the model

In [461]:
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, stateful=True, batch_input_shape=(1,None,input_dim)),
    tf.keras.layers.SimpleRNN(32, return_sequences=True, stateful=True),
    tf.keras.layers.SimpleRNN(16, return_sequences=True, stateful=True),
    tf.keras.layers.SimpleRNN(16, return_sequences=False, stateful=True),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='linear') # Adjust based on your output requirements
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.build()
model.compile()

In [462]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (1, None, 32)             1856      
                                                                 
 simple_rnn_1 (SimpleRNN)    (1, None, 32)             2080      
                                                                 
 simple_rnn_2 (SimpleRNN)    (1, None, 16)             784       
                                                                 
 simple_rnn_3 (SimpleRNN)    (1, 16)                   528       
                                                                 
 dense_59 (Dense)            (1, 32)                   544       
                                                                 
 dense_60 (Dense)            (1, 16)                   528       
                                                                 
 dense_61 (Dense)            (1, 16)                 

### Helper functions

In [312]:
def generateRandomBoolArr(length, numOfOnes):
    result = np.zeros(length, dtype=int)
    indices = np.random.choice(length, numOfOnes, replace=False)
    result[indices] = 1
    return result

In [313]:
def modelOutToPosition(modelOut, allowedPVs, oldPosition):
    for i, change in enumerate(modelOut):
        if allowedPVs[i]:
            oldPosition[usablePVindextable[i]] += change
        modelOut[i] = oldPosition[usablePVindextable[i]]
    return [oldPosition, modelOut]

In [314]:
def biasedRandomNumber(max_value):
    prob = np.arange(1, max_value + 1)
    # Normalize probabilities to sum to 1
    prob = prob / np.sum(prob)
    result = max_value + 1 - np.random.choice(np.arange(1, max_value + 1), p=prob)
    return result

In [311]:
#40104 is the index for the global optimum
optimum = list(sim.states[40104][0])
baseOptimum = []

for i in usablePVindextable:
    baseOptimum.append(optimum[i])

var_I1 = [0.4656249999999993,0.17000000000000035,0.6524999999999999,0.14566250000000042,0.34001250000000066,0.19999999999999996,0.14499999999999957,0.13628749999999912,2.239959716796875,1.40057373046875,2.3402099609375,1.9005126953125]

def get_StartingPossition():
    sectionPos = []
    for i, variance in enumerate(var_I1):
        newValue = baseOptimum[i] + (random.random()-0.5) * variance
        optimum[usablePVindextable[i]] = newValue
        sectionPos.append(newValue)
    return [np.array(optimum), np.array(sectionPos)]

count = 0
for i in range(1000):
    get_StartingPossition()
    if sim.get(optimum) < 0:
        count += 1
print(count/1000)

0.529


## Trainig Loop

In [471]:
# Training loop
runsPerepisode = 100
episodes = 4000

bestLoss = 0
bestDec = 0

num_steps = 4.9
print("Starting Traning...")
for episode in range(episodes):
    
    num_steps = min(num_steps + 0.01, 30)
    sumList = [0,0,0]
    for run in range(runsPerepisode):
        # This is one run
        
        # Reset the LSTM states
        model.layers[0].reset_states()
        model.layers[1].reset_states() 
        model.layers[2].reset_states()
        model.layers[3].reset_states()
        
        # Choose a starting position
        position, sectionPos = get_StartingPossition()
        
        # Choose which PVs we can manipulate in this run
        allowedPVs = generateRandomBoolArr(output_dim, 12)
        
        # Init loss
        y = sim.get(position)
        minY = y
        startY = y
        
        states = []
        returns = []
        action = []
        fail = 0
        with tf.GradientTape() as tape:
            # Forward pass
            for step in range(math.floor(num_steps)):

                modelIn = np.reshape(np.concatenate((allowedPVs, sectionPos, [y]), axis=0), (1, 1, input_dim))
                modelOut = model(modelIn)
                action.append(modelOut)
                # We now have our next step
                position , sectionPos = modelOutToPosition(modelOut.numpy()[0], allowedPVs, position)
                # We can calculate the loss right here
                lastY = y
                y = sim.get(position)
                returns.append((min(y-minY, 0) + (y-lastY))/ math.floor(num_steps))
                
                
            
                if y < minY: 
                    minY = y
            #We made a run Now to calculate a gradient
            result = [(action * allowedPVs/ tf.reduce_sum(abs(action * allowedPVs))) * returns for action, returns in zip(action, returns)]
            loss = tf.reduce_sum(result) 

        # Backward pass
        gradients = tape.gradient(tf.constant(loss), model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))   
        sumList[0] += loss
        sumList[1] += minY - startY
        if startY != minY:
            sumList[2] += 1

    # Print episode information
    print(f'episode {episode+1}/{episodes}, AVG Loss: {sumList[0]/runsPerepisode: .3f}, AVG improvment: {sumList[1]/runsPerepisode: .3f}, Acc: {(sumList[2]/runsPerepisode)*100: .1f}%')
    
    if sumList[0]/runsPerepisode < bestLoss:
        bestLoss = sumList[0]/runsPerepisode
        print(f'------------------------ New best Loss with {bestLoss: .3f} ------------------------')
        model.save('BestLossRNN.keras')
    
    if (sumList[1]/runsPerepisode) * (sumList[2]/runsPerepisode) < bestDec:
        bestDec = (sumList[1]/runsPerepisode) * (sumList[2]/runsPerepisode)
        print(f'------------------------ New best Dec with {bestDec: .3f} ------------------------')
        model.save('BestDecRNN.keras')
    
    if episode % 100 == 0:
        name = f'ModelRNNE{episode}_L{sumList[0]/runsPerepisode: .2f}_I{sumList[1]/runsPerepisode: .2f}_P{sumList[2]/runsPerepisode: .2f}'
        print("Modelsnapshot saved as:" + name)
        model.save(name + '.keras')
        

Starting Traning...
Epoch 1/4000, AVG Loss:  6.085, AVG improvment: -15.062, Acc:  32.0%
------------------------ New best Dec with -4.820 ------------------------
Modelsnapshot saved as:ModelRNNE0_L 6.09_I-15.06_P 0.32
Epoch 2/4000, AVG Loss:  8.556, AVG improvment: -18.853, Acc:  33.0%
------------------------ New best Dec with -6.221 ------------------------
Epoch 3/4000, AVG Loss:  9.223, AVG improvment: -11.643, Acc:  29.0%
Epoch 4/4000, AVG Loss:  10.558, AVG improvment: -14.238, Acc:  25.0%
Epoch 5/4000, AVG Loss:  6.710, AVG improvment: -19.881, Acc:  35.0%
------------------------ New best Dec with -6.958 ------------------------
Epoch 6/4000, AVG Loss:  4.264, AVG improvment: -22.362, Acc:  42.0%
------------------------ New best Dec with -9.392 ------------------------
Epoch 7/4000, AVG Loss:  11.310, AVG improvment: -13.590, Acc:  28.0%
Epoch 8/4000, AVG Loss:  10.756, AVG improvment: -12.715, Acc:  21.0%
Epoch 9/4000, AVG Loss:  9.413, AVG improvment: -11.481, Acc:  22.0%


Epoch 111/4000, AVG Loss:  7.360, AVG improvment: -10.160, Acc:  23.0%
Epoch 112/4000, AVG Loss:  6.332, AVG improvment: -12.541, Acc:  29.0%
Epoch 113/4000, AVG Loss:  4.618, AVG improvment: -17.044, Acc:  28.0%
Epoch 114/4000, AVG Loss:  5.257, AVG improvment: -17.318, Acc:  21.0%
Epoch 115/4000, AVG Loss:  4.527, AVG improvment: -15.094, Acc:  20.0%
Epoch 116/4000, AVG Loss:  3.590, AVG improvment: -16.982, Acc:  33.0%
Epoch 117/4000, AVG Loss:  6.709, AVG improvment: -10.815, Acc:  28.0%
Epoch 118/4000, AVG Loss:  8.819, AVG improvment: -11.911, Acc:  28.0%
Epoch 119/4000, AVG Loss:  5.920, AVG improvment: -10.558, Acc:  20.0%
Epoch 120/4000, AVG Loss:  7.984, AVG improvment: -13.186, Acc:  26.0%
Epoch 121/4000, AVG Loss:  6.005, AVG improvment: -14.536, Acc:  20.0%
Epoch 122/4000, AVG Loss:  5.076, AVG improvment: -17.264, Acc:  27.0%
Epoch 123/4000, AVG Loss:  7.687, AVG improvment: -15.203, Acc:  24.0%
Epoch 124/4000, AVG Loss:  4.088, AVG improvment: -12.705, Acc:  25.0%
Epoch 

Epoch 226/4000, AVG Loss:  6.341, AVG improvment: -9.781, Acc:  28.0%
Epoch 227/4000, AVG Loss:  5.024, AVG improvment: -15.290, Acc:  33.0%
Epoch 228/4000, AVG Loss:  4.808, AVG improvment: -18.919, Acc:  36.0%
Epoch 229/4000, AVG Loss:  5.427, AVG improvment: -14.363, Acc:  29.0%
Epoch 230/4000, AVG Loss:  5.198, AVG improvment: -16.964, Acc:  33.0%
Epoch 231/4000, AVG Loss:  5.894, AVG improvment: -16.490, Acc:  33.0%
Epoch 232/4000, AVG Loss:  6.196, AVG improvment: -14.024, Acc:  26.0%
Epoch 233/4000, AVG Loss:  5.229, AVG improvment: -10.317, Acc:  20.0%
Epoch 234/4000, AVG Loss:  5.024, AVG improvment: -12.711, Acc:  27.0%
Epoch 235/4000, AVG Loss:  6.494, AVG improvment: -9.282, Acc:  25.0%
Epoch 236/4000, AVG Loss:  6.220, AVG improvment: -17.939, Acc:  33.0%
Epoch 237/4000, AVG Loss:  3.932, AVG improvment: -15.084, Acc:  29.0%
Epoch 238/4000, AVG Loss:  6.659, AVG improvment: -11.840, Acc:  24.0%
Epoch 239/4000, AVG Loss:  5.579, AVG improvment: -11.953, Acc:  30.0%
Epoch 24

Epoch 338/4000, AVG Loss:  5.035, AVG improvment: -19.030, Acc:  42.0%
Epoch 339/4000, AVG Loss:  3.940, AVG improvment: -20.026, Acc:  34.0%
Epoch 340/4000, AVG Loss:  4.903, AVG improvment: -20.243, Acc:  45.0%
Epoch 341/4000, AVG Loss:  5.294, AVG improvment: -19.347, Acc:  41.0%
Epoch 342/4000, AVG Loss:  5.798, AVG improvment: -10.892, Acc:  30.0%
Epoch 343/4000, AVG Loss:  5.959, AVG improvment: -13.554, Acc:  36.0%
Epoch 344/4000, AVG Loss:  4.423, AVG improvment: -14.162, Acc:  37.0%
Epoch 345/4000, AVG Loss:  6.272, AVG improvment: -13.855, Acc:  36.0%
Epoch 346/4000, AVG Loss:  3.693, AVG improvment: -21.090, Acc:  32.0%
Epoch 347/4000, AVG Loss:  4.930, AVG improvment: -13.398, Acc:  36.0%
Epoch 348/4000, AVG Loss:  4.830, AVG improvment: -10.400, Acc:  27.0%
Epoch 349/4000, AVG Loss:  4.199, AVG improvment: -21.206, Acc:  40.0%
Epoch 350/4000, AVG Loss:  2.918, AVG improvment: -23.899, Acc:  38.0%
Epoch 351/4000, AVG Loss:  3.711, AVG improvment: -20.439, Acc:  31.0%
Epoch 

Epoch 452/4000, AVG Loss:  4.060, AVG improvment: -16.823, Acc:  38.0%
Epoch 453/4000, AVG Loss:  3.552, AVG improvment: -17.002, Acc:  39.0%
Epoch 454/4000, AVG Loss:  4.554, AVG improvment: -7.231, Acc:  28.0%
Epoch 455/4000, AVG Loss:  5.351, AVG improvment: -12.709, Acc:  38.0%
Epoch 456/4000, AVG Loss:  4.604, AVG improvment: -15.992, Acc:  37.0%
Epoch 457/4000, AVG Loss:  4.568, AVG improvment: -15.125, Acc:  36.0%
Epoch 458/4000, AVG Loss:  3.334, AVG improvment: -12.296, Acc:  30.0%
Epoch 459/4000, AVG Loss:  3.998, AVG improvment: -18.748, Acc:  33.0%
Epoch 460/4000, AVG Loss:  5.741, AVG improvment: -14.477, Acc:  26.0%
Epoch 461/4000, AVG Loss:  2.850, AVG improvment: -22.062, Acc:  35.0%
Epoch 462/4000, AVG Loss:  5.511, AVG improvment: -12.403, Acc:  23.0%
Epoch 463/4000, AVG Loss:  3.810, AVG improvment: -9.797, Acc:  22.0%
Epoch 464/4000, AVG Loss:  4.786, AVG improvment: -13.372, Acc:  29.0%
Epoch 465/4000, AVG Loss:  5.033, AVG improvment: -14.712, Acc:  26.0%
Epoch 46

Epoch 567/4000, AVG Loss:  3.055, AVG improvment: -15.502, Acc:  39.0%
Epoch 568/4000, AVG Loss:  4.706, AVG improvment: -9.346, Acc:  31.0%
Epoch 569/4000, AVG Loss:  4.776, AVG improvment: -12.202, Acc:  36.0%
Epoch 570/4000, AVG Loss:  3.554, AVG improvment: -7.296, Acc:  27.0%
Epoch 571/4000, AVG Loss:  4.379, AVG improvment: -6.032, Acc:  27.0%
Epoch 572/4000, AVG Loss:  3.378, AVG improvment: -8.347, Acc:  34.0%
Epoch 573/4000, AVG Loss:  3.641, AVG improvment: -9.752, Acc:  34.0%
Epoch 574/4000, AVG Loss:  4.466, AVG improvment: -5.641, Acc:  25.0%
Epoch 575/4000, AVG Loss:  4.937, AVG improvment: -8.798, Acc:  30.0%
Epoch 576/4000, AVG Loss:  4.579, AVG improvment: -4.929, Acc:  21.0%
Epoch 577/4000, AVG Loss:  4.012, AVG improvment: -8.847, Acc:  35.0%
Epoch 578/4000, AVG Loss:  3.903, AVG improvment: -4.300, Acc:  25.0%
Epoch 579/4000, AVG Loss:  5.127, AVG improvment: -6.374, Acc:  26.0%
Epoch 580/4000, AVG Loss:  5.714, AVG improvment: -6.630, Acc:  28.0%
Epoch 581/4000, AV

Epoch 684/4000, AVG Loss:  3.483, AVG improvment: -6.447, Acc:  13.0%
Epoch 685/4000, AVG Loss:  4.235, AVG improvment: -6.701, Acc:  14.0%
Epoch 686/4000, AVG Loss:  3.902, AVG improvment: -4.289, Acc:  19.0%
Epoch 687/4000, AVG Loss:  4.168, AVG improvment: -8.183, Acc:  28.0%
Epoch 688/4000, AVG Loss:  3.818, AVG improvment: -7.697, Acc:  20.0%
Epoch 689/4000, AVG Loss:  5.252, AVG improvment: -3.874, Acc:  13.0%
Epoch 690/4000, AVG Loss:  4.899, AVG improvment: -6.049, Acc:  21.0%
Epoch 691/4000, AVG Loss:  2.783, AVG improvment: -5.985, Acc:  16.0%
Epoch 692/4000, AVG Loss:  3.754, AVG improvment: -11.900, Acc:  22.0%
Epoch 693/4000, AVG Loss:  3.818, AVG improvment: -4.831, Acc:  13.0%
Epoch 694/4000, AVG Loss:  4.088, AVG improvment: -5.053, Acc:  15.0%


KeyboardInterrupt: 